In [398]:
import pandas as pd
import numpy as np
import astropy.utils.data
from astropy.coordinates import SkyCoord,concatenate
import astropy.units as u
import datetime as _datetime
from astropy.io import fits as _fits
from astropy.coordinates import FK5,ICRS
from astropy.io import ascii
from astropy.coordinates import solar_system_ephemeris
from astropy.time import Time

# Join results tables together

In [399]:
df_200219=pd.read_csv('200219_results.csv')
df_191213=pd.read_csv('191213g_results.csv')
final_results=pd.concat([df_200219,df_191213], axis=0)
final_results=final_results.reset_index(drop=True)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,3,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,3,4,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [400]:
final_results_ps1=final_results[final_results['source']=='PS1']
final_results_glade=final_results[final_results['source']=='GLADE']

final_results_ps1[['our_id','RA','Dec']].to_csv('ps1_sources_for_gaia.csv')
final_results_glade[['our_id','RA','Dec']].to_csv('glade_sources_for_gaia.csv')
final_results[['our_id','source','RA','Dec']].to_csv('all_sources_for_gaia.csv')


In [401]:
print(list(final_results.columns))

['Unnamed: 0', 'PGC', 'GWGC', 'HyperLEDA', '2MASS', 'SDSS', 'flag1', 'RA', 'Dec', 'dist', 'dist_err', 'z', 'Bmag', 'e_Bmag', 'AbsBMag', 'Jmag', 'e_Jmag', 'Hmag', 'e_Hmag', 'Kmag', 'eKmag', 'flag2', 'flag3', 'source', 'objName', 'objNameHMS', 'objAltName1', 'objAltName2', 'objAltName3', 'objPopularName', 'objID', 'uniquePspsOBid', 'ippObjID', 'surveyID', 'htmID', 'zoneID', 'tessID', 'projectionID', 'skyCellID', 'randomID', 'batchID', 'dvoRegionID', 'processingVersion', 'objInfoFlag', 'qualityFlag', 'raStack', 'decStack', 'raStackErr', 'decStackErr', 'raMeanErr', 'decMeanErr', 'epochMean', 'posMeanChisq', 'cx', 'cy', 'cz', 'lambda', 'beta', 'l', 'b', 'nStackObjectRows', 'nStackDetections', 'nDetections', 'ng', 'nr', 'ni', 'nz', 'ny', 'Column1', 'Column2', 'gQfPerfect', 'gMeanPSFMag', 'gMeanPSFMagErr', 'gMeanPSFMagStd', 'gMeanPSFMagNpt', 'gMeanPSFMagMin', 'gMeanPSFMagMax', 'gMeanKronMag', 'gMeanKronMagErr', 'gMeanKronMagStd', 'gMeanKronMagNpt', 'gMeanApMag', 'gMeanApMagErr', 'gMeanApMagSt

## Join PS1 crossmatch with GLADE sources

In [402]:
glade_ps1_crossmatch=pd.read_csv('gladecheck1arsec2_Lana_S.csv')
glade_ps1_crossmatch['Dec']=glade_ps1_crossmatch['DEC']


In [403]:
print(list(glade_ps1_crossmatch.columns))

['ID', 'our_id', 'RA', 'DEC', 'ObjID', 'Column1', 'Column2', 'cx', 'cy', 'cz', 'htmID', 'distance', 'objName', 'objNameHMS', 'objAltName1', 'objAltName2', 'objAltName3', 'objPopularName', 'Column3', 'uniquePspsOBid', 'ippObjID', 'surveyID', 'Column4', 'zoneID', 'tessID', 'projectionID', 'skyCellID', 'randomID', 'batchID', 'dvoRegionID', 'processingVersion', 'objInfoFlag', 'qualityFlag', 'raStack', 'decStack', 'raStackErr', 'decStackErr', 'raMean', 'decMean', 'raMeanErr', 'decMeanErr', 'epochMean', 'posMeanChisq', 'Column5', 'Column6', 'Column7', 'lambda', 'beta', 'l', 'b', 'nStackObjectRows', 'nStackDetections', 'nDetections', 'ng', 'nr', 'ni', 'nz', 'ny', 'Column8', 'Column9', 'gQfPerfect', 'gMeanPSFMag', 'gMeanPSFMagErr', 'gMeanPSFMagStd', 'gMeanPSFMagNpt', 'gMeanPSFMagMin', 'gMeanPSFMagMax', 'gMeanKronMag', 'gMeanKronMagErr', 'gMeanKronMagStd', 'gMeanKronMagNpt', 'gMeanApMag', 'gMeanApMagErr', 'gMeanApMagStd', 'gMeanApMagNpt', 'gFlags', 'rQfPerfect', 'rMeanPSFMag', 'rMeanPSFMagErr',

In [404]:
def sep(ra,dec,ra1,dec1):
    #c1 = SkyCoord(ra=ra, '-69d45m22s', frame='icrs')
    c1=SkyCoord(ra*u.deg, dec*u.deg)
    c2=SkyCoord(ra1*u.deg, dec1*u.deg)
    sep = c1.separation(c2)
    return sep.arcsecond

glade_ps1_crossmatch['separation']=np.abs(sep(glade_ps1_crossmatch['RA'].values,glade_ps1_crossmatch['Dec'].values,glade_ps1_crossmatch['raMean'].values,glade_ps1_crossmatch['decMean'].values))
glade_ps1_crossmatch=glade_ps1_crossmatch.sort_values('separation').drop_duplicates(subset='our_id', keep='first')

#glade_ps1_crossmatch['RA']=xmatch_glade_ps1_cut['ra']
#xmatch_glade_ps1_cut['Dec']=xmatch_glade_ps1_cut['dec']

In [405]:
# merge PS1 and glade crossmatch columns
#final_results = final_results.merge(glade_ps1_crossmatch, how = 'outer', on = ['our_id', 'our_id'])

df1 = final_results.set_index(["our_id"])
df2 = glade_ps1_crossmatch.set_index(["our_id"])

final_results = df1.combine_first(df2).reset_index()
#final_results=pd.concat([final_results,glade_ps1_crossmatch])

In [406]:
print(len(final_results))

69939


# Load in xmatch

In [407]:
xmatch_ps1_only=pd.read_csv('gaia_ps1_xmatch_result.csv')
xmatch_all=pd.read_csv('all_xmatch_dr3.csv')


df1 = xmatch_all.set_index(["our_id"])
df2 = glade_ps1_crossmatch.set_index(["our_id"])

xmatch_all = df1.combine_first(df2).reset_index()

In [408]:
print(list(xmatch_all.columns))
print(len(xmatch_all))

['our_id', 'Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'DEC', 'Dec', 'ID', 'ObjID', 'RA', 'all_sources_for_gaia_oid', 'astrometric_chi2_al', 'astrometric_excess_noise', 'astrometric_excess_noise_sig', 'astrometric_gof_al', 'astrometric_matched_transits', 'astrometric_n_bad_obs_al', 'astrometric_n_good_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_obs_al', 'astrometric_params_solved', 'astrometric_primary_flag', 'astrometric_sigma5d_max', 'b', 'batchID', 'beta', 'bp_g', 'bp_rp', 'col1', 'cx', 'cy', 'cz', 'dec', 'dec.1', 'decMean', 'decMeanErr', 'decStack', 'decStackErr', 'dec_error', 'dec_parallax_corr', 'dec_pmdec_corr', 'dec_pmra_corr', 'dec_pseudocolour_corr', 'designation', 'dist', 'distance', 'dr2_radial_velocity', 'dr2_radial_velocity_error', 'dr2_rv_nb_transits', 'dr2_rv_template_fe_h', 'dr2_rv_template_logg', 'dr2_rv_template_teff', 'duplicated_source', 'dvoRegionID', 'ecl_lat', 'ecl_lon', 'epochMean', 'gFlags', 'gMeanApMa

# Identify sources not crossmatched

In [409]:
common = final_results.merge(xmatch_all,on=['our_id','our_id'])

not_xmatched=final_results[(~final_results.our_id.isin(common.our_id))&(~final_results.our_id.isin(common.our_id))]
print(len(not_xmatched))

14746


# Cut on astrometric error and proper motion

In [410]:
xmatch_all['uwe']=np.sqrt(xmatch_all['astrometric_chi2_al']/(xmatch_all['astrometric_n_good_obs_al']-5))
print(min(xmatch_all['uwe']), max(xmatch_all['uwe']), np.nanmin(xmatch_all['pmra']), np.nanmax(xmatch_all['pmdec']))
#and cut on pmra pmdec


0.8511615078820235 166.74739273060956 -75.2187536114139 69.89171461007075


In [411]:
print(np.mean(xmatch_all['uwe']),np.mean(xmatch_all['pmra']))

2.2004502193010933 -2.2504660458717862


In [412]:
xmatch_ps1_only=xmatch_all[xmatch_all['source']=='PS1']
xmatch_GLADE_only=xmatch_all[xmatch_all['source']=='GLADE']

xmatch_cut=xmatch_ps1_only[(xmatch_ps1_only['uwe']>1.5) & (np.abs(xmatch_ps1_only['pmra']<3) | (xmatch_ps1_only['pmra']==np.nan)) & (np.abs(xmatch_ps1_only['pmdec']<3) | (xmatch_ps1_only['pmdec']==np.nan))]
print(len(xmatch_cut))

2640


# Recombine gaia-limited df with ps1/GLADE columns

In [413]:
xmatch_glade_ps1_cut=pd.concat([xmatch_cut,xmatch_GLADE_only], axis=0)
print(len(xmatch_glade_ps1_cut))

11067


# If there are duplicate PS1 crossmatches, only take closest one

In [414]:
def sep(ra,dec,ra1,dec1):
    #c1 = SkyCoord(ra=ra, '-69d45m22s', frame='icrs')
    c1=SkyCoord(ra*u.deg, dec*u.deg)
    c2=SkyCoord(ra1*u.deg, dec1*u.deg)
    sep = c1.separation(c2)
    return sep.arcsecond

xmatch_glade_ps1_cut['separation']=np.abs(sep(xmatch_glade_ps1_cut['ra'].values,xmatch_glade_ps1_cut['dec'].values,xmatch_glade_ps1_cut['ra.1'].values,xmatch_glade_ps1_cut['dec.1'].values))
xmatch_glade_ps1_cut=xmatch_glade_ps1_cut.sort_values('separation').drop_duplicates(subset='our_id', keep='first')

xmatch_glade_ps1_cut['RA']=xmatch_glade_ps1_cut['ra']
xmatch_glade_ps1_cut['Dec']=xmatch_glade_ps1_cut['dec']
#

# Merge xmatch with non-xmatch

In [415]:
total_cut_df = pd.concat([not_xmatched,xmatch_glade_ps1_cut])

In [416]:
print(list(total_cut_df.columns))
print(len(total_cut_df))

['our_id', '2MASS', 'AbsBMag', 'Bmag', 'Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'DEC', 'Dec', 'GWGC', 'Hmag', 'HyperLEDA', 'ID', 'Jmag', 'Kmag', 'ObjID', 'PGC', 'RA', 'SDSS', 'Unnamed: 0', 'b', 'batchID', 'beta', 'cx', 'cy', 'cz', 'decMean', 'decMeanErr', 'decStack', 'decStackErr', 'dist', 'dist_err', 'distance', 'dvoRegionID', 'eKmag', 'e_Bmag', 'e_Hmag', 'e_Jmag', 'epochMean', 'flag1', 'flag2', 'flag3', 'gFlags', 'gMeanApMag', 'gMeanApMagErr', 'gMeanApMagNpt', 'gMeanApMagStd', 'gMeanKronMag', 'gMeanKronMagErr', 'gMeanKronMagNpt', 'gMeanKronMagStd', 'gMeanPSFMag', 'gMeanPSFMagErr', 'gMeanPSFMagMax', 'gMeanPSFMagMin', 'gMeanPSFMagNpt', 'gMeanPSFMagStd', 'gQfPerfect', 'htmID', 'iFlags', 'iMeanApMag', 'iMeanApMagErr', 'iMeanApMagNpt', 'iMeanApMagStd', 'iMeanKronMag', 'iMeanKronMagErr', 'iMeanKronMagNpt', 'iMeanKronMagStd', 'iMeanPSFMag', 'iMeanPSFMagErr', 'iMeanPSFMagMax', 'iMeanPSFMagMin', 'iMeanPSFMagNpt', 'iMeanPSFMagStd', 'iQf

In [437]:
print(total_cut_df[['our_id', 'source','gMeanPSFMag','ra.1','RA','ra','dec']])

             our_id source  gMeanPSFMag        ra.1          RA          ra  \
0      191213_10004    PS1    21.553801         NaN  252.269439         NaN   
1      191213_10012    PS1    20.655100         NaN  250.872067         NaN   
2      191213_10015    PS1    21.074400         NaN  252.275868         NaN   
3      191213_10016    PS1    20.662201         NaN  251.347515         NaN   
4      191213_10017    PS1    20.511600         NaN  252.440333         NaN   
...             ...    ...          ...         ...         ...         ...   
25656   191213_3967  GLADE    18.084700  297.107849  297.108142  297.108142   
25657    200129_946  GLADE    18.790100  319.651000  319.650723  319.650723   
25658   191213_1617  GLADE    24.242001  104.980667  104.980718  104.980718   
25659   200129_3680  GLADE          NaN  319.433000  319.433277  319.433277   
25660   200129_3118  GLADE          NaN  318.125000  318.125282  318.125282   

             dec  
0            NaN  
1            

# Template FITS

In [418]:
template_hdulist = _fits.open('WS2021A1-003_CatalogueTemplate.fits')
template_primary_hdu = template_hdulist[0]
template_hdu = template_hdulist[1]
template_column_names = [col.name for col in template_hdu.columns]
print(template_column_names)


['CNAME', 'TARGSRVY', 'TARGPROG', 'TARGCAT', 'TARGID', 'TARGNAME', 'TARGPRIO', 'TARGUSE', 'TARGCLASS', 'PROGTEMP', 'OBSTEMP', 'GAIA_ID', 'GAIA_DR', 'GAIA_RA', 'GAIA_DEC', 'GAIA_EPOCH', 'GAIA_PMRA', 'GAIA_PMRA_ERR', 'GAIA_PMDEC', 'GAIA_PMDEC_ERR', 'GAIA_PARAL', 'GAIA_PARAL_ERR', 'HEALPIX', 'IFU_SPAXEL', 'IFU_PA', 'IFU_DITHER', 'MAG_G', 'MAG_G_ERR', 'MAG_R', 'MAG_R_ERR', 'MAG_I', 'MAG_I_ERR', 'GAIA_MAG_G', 'GAIA_MAG_G_ERR', 'GAIA_MAG_BP', 'GAIA_MAG_BP_ERR', 'GAIA_MAG_RP', 'GAIA_MAG_RP_ERR', 'APS_WL_MIN', 'APS_WL_MAX', 'APS_Z', 'APS_SIGMA', 'APS_TEMPL_LIB', 'APS_TEMPL_LIB_NORM', 'APS_PPXF_WL_MIN', 'APS_PPXF_WL_MAX', 'APS_PPXF_MOM', 'APS_PPXF_DEG_ADD', 'APS_PPXF_DEG_MULT', 'APS_PPXF_NUM_MC', 'APS_GAND_MODE', 'APS_GAND_ERR', 'APS_GAND_RED1', 'APS_GAND_RED2', 'APS_GAND_EBV', 'APS_LS_MODE', 'APS_LS_RES', 'APS_LS_NUM_MC', 'APS_SSP_NUM_WLKR', 'APS_SSP_NUM_CHAIN', 'APS_IFU_MASK', 'APS_IFU_TSSL_TYPE', 'APS_IFU_TSSL_TARG_SNR', 'APS_IFU_TSSL_MIN_SNR', 'APS_IFU_TSSL_COVAR', 'APS_IFU_SRC_ID', 'APS_IF

In [419]:
from astropy.table import Table
Table.read(template_hdulist, hdu=1)

CNAME,TARGSRVY,TARGPROG,TARGCAT,TARGID,TARGNAME,TARGPRIO,TARGUSE,TARGCLASS,PROGTEMP,OBSTEMP,GAIA_ID,GAIA_DR,GAIA_RA,GAIA_DEC,GAIA_EPOCH,GAIA_PMRA,GAIA_PMRA_ERR,GAIA_PMDEC,GAIA_PMDEC_ERR,GAIA_PARAL,GAIA_PARAL_ERR,HEALPIX,IFU_SPAXEL,IFU_PA,IFU_DITHER,MAG_G,MAG_G_ERR,MAG_R,MAG_R_ERR,MAG_I,MAG_I_ERR,GAIA_MAG_G,GAIA_MAG_G_ERR,GAIA_MAG_BP,GAIA_MAG_BP_ERR,GAIA_MAG_RP,GAIA_MAG_RP_ERR,APS_WL_MIN,APS_WL_MAX,APS_Z,APS_SIGMA,APS_TEMPL_LIB,APS_TEMPL_LIB_NORM,APS_PPXF_WL_MIN,APS_PPXF_WL_MAX,APS_PPXF_MOM,APS_PPXF_DEG_ADD,APS_PPXF_DEG_MULT,APS_PPXF_NUM_MC,APS_GAND_MODE,APS_GAND_ERR,APS_GAND_RED1,APS_GAND_RED2,APS_GAND_EBV,APS_LS_MODE,APS_LS_RES,APS_LS_NUM_MC,APS_SSP_NUM_WLKR,APS_SSP_NUM_CHAIN,APS_IFU_MASK,APS_IFU_TSSL_TYPE,APS_IFU_TSSL_TARG_SNR,APS_IFU_TSSL_MIN_SNR,APS_IFU_TSSL_COVAR,APS_IFU_SRC_ID,APS_IFU_SRC_RA,APS_IFU_SRC_DEC,APS_FLAG,NIRCAT,NIRCAT_DEC,NIRCAT_DR,NIRCAT_ID,NIRCAT_MAG_H,NIRCAT_MAG_H_ERR,NIRCAT_MAG_J,NIRCAT_MAG_J_ERR,NIRCAT_MAG_K,NIRCAT_MAG_K_ERR,NIRCAT_RA,OPTCAT,OPTCAT_DEC,OPTCAT_DR,OPTCAT_ID,OPTCAT_MAG_B,OPTCAT_MAG_B_ERR,OPTCAT_RA,PS_DEC,PS_DR,PS_ID,PS_MAG_G,PS_MAG_G_ERR,PS_MAG_I,PS_MAG_I_ERR,PS_MAG_R,PS_MAG_R_ERR,PS_MAG_Y,PS_MAG_Y_ERR,PS_MAG_Z,PS_MAG_Z_ERR,PS_RA
,,,,,,,,,,,,,deg,deg,yr,mas / yr,mas / yr,mas / yr,mas / yr,mas,mas,,,deg,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,Angstrom,Angstrom,,km / s,,,Angstrom,Angstrom,,,,,,,,,,,Angstrom,,,,,,,,,,deg,deg,,,deg,,,mag,mag,mag,mag,mag,mag,deg,,deg,,,mag,mag,deg,deg,,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,deg
str20,str15,str40,str30,str30,str30,float32,str1,str12,str8,str5,str25,str5,float64,float64,float32,float32,float32,float32,float32,float32,float32,int16,str6,float64,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str10,int16,float32,float32,int16,int16,int16,int16,int16,int16,float32,float32,int16,int16,float32,int16,int16,int16,int16,str10,float32,float32,int16,str35,float64,float64,str48,str50,float64,str10,str30,float32,float32,float32,float32,float32,float32,float64,str60,float64,str50,str30,float32,float32,float64,float64,str10,str30,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64


## Extension 0 Header

In [420]:
ext0_header=template_hdulist[0].header

#Set the catalogue submitter and CCs

ext0_header['CAT_NME1']='Lana'
ext0_header['CAT_NME2']='Salmon'
ext0_header['CAT_MAIL']='lana.salmon@ucdconnect.ie'
ext0_header['CAT_CC']='p.jonker@sron.nl,morgan.fraser@ucd.ie,lorraine.hanlon@ucd.ie,antonio.martin-carrillo@ucd.ie'
print(repr(ext0_header))



SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
COMMENT WS2021A1-003 Catalogue Template                                         
DATAMVER= '8.00    '           / WEAVE Data Model Version                       
TRIMESTE= '2021A1  '           / Observing Trimester                            
MAG_G_CM= '' / Survey specific mag column(s) used for MAG_G                     
MAG_R_CM= '' / Survey specific mag column(s) used for MAG_R                     
MAG_I_CM= '' / Survey specific mag column(s) used for MAG_I                     
STL_NME1= 'Peter   '                                                            
STL_NME2= 'Jonker  '                                                            
STL_MAIL= 'p.jonker@sron.nl'

## Extension 1 Header

In [421]:
ext1_header=template_hdulist[1].header
print(repr(ext1_header))

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  916 / length of dimension 1                          
NAXIS2  =                    0 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                  101 / number of table fields                         
EXTNAME = 'WS2021A1-003 CATALOGUE' / extension name                             
TTYPE1  = 'CNAME   '           / WEAVE object name from coordinates             
TFORM1  = '20A     '           / data format of field: ASCII Character          
TDISP1  = 'A20     '           / Display format for column                      
TUCD1   = 'meta.id;meta.main

## Calculate gaia mag errors

In [422]:
# according to dr2 https://www.cosmos.esa.int/documents/29201/1773953/Gaia+DR2+primer+version+1.3.pdf/a4459741-6732-7a98-1406-a1bea243df79?t=1581668739161
total_cut_df['phot_g_mean_mag_err']=1.086/total_cut_df['phot_g_mean_flux_over_error']
total_cut_df['phot_bp_mean_mag_err']=1.086/total_cut_df['phot_bp_mean_flux_over_error']
total_cut_df['phot_rp_mean_mag_err']=1.086/total_cut_df['phot_rp_mean_flux_over_error']

## Take relevant columns from our catalogue

In [461]:
total_cut_df = total_cut_df.reset_index(drop=True)
FITS_table_from_cat=total_cut_df[['source','our_id','source_id','ra.1', 'dec.1', 'ref_epoch', 'pmra', 'pmra_error',
                                  'pmdec', 'pmdec_error','parallax', 'parallax_error','phot_g_mean_mag',
                                  'phot_g_mean_mag_err','phot_bp_mean_mag','phot_bp_mean_mag_err',
                                  'phot_rp_mean_mag','phot_rp_mean_mag_err', 'gMeanKronMag', 'gMeanKronMagErr',
                                  'rMeanKronMag', 'rMeanKronMagErr','iMeanKronMag', 'iMeanKronMagErr',
                                  'objID', 'raMean', 'decMean',
                                  'gMeanKronMag', 'gMeanKronMagErr','iMeanKronMag', 'iMeanKronMagErr',
                                  'rMeanKronMag', 'rMeanKronMagErr','yMeanKronMag', 'yMeanKronMagErr','zMeanKronMag', 'zMeanKronMagErr',
                                  'ra', 'dec', 'SDSS', 'Hmag', 'e_Hmag','Jmag', 'e_Jmag',  'Kmag', 'eKmag',
                                  'ra', 'dec', 'SDSS','Bmag', 'e_Bmag'
                                ]]


FITS_table_from_cat.columns = ['COORDCAT','TARGID','GAIA_ID','GAIA_RA', 'GAIA_DEC', 'GAIA_EPOCH', 'GAIA_PMRA', 'GAIA_PMRA_ERR', 
                               'GAIA_PMDEC', 'GAIA_PMDEC_ERR', 'GAIA_PARAL', 'GAIA_PARAL_ERR','GAIA_MAG_G', 
                               'GAIA_MAG_G_ERR', 'GAIA_MAG_BP', 'GAIA_MAG_BP_ERR', 
                               'GAIA_MAG_RP', 'GAIA_MAG_RP_ERR', 'MAG_G', 'MAG_G_ERR', 
                               'MAG_R', 'MAG_R_ERR', 'MAG_I', 'MAG_I_ERR',
                               'PS_ID', 'PS_RA', 'PS_DEC',
                               'PS_MAG_G', 'PS_MAG_G_ERR', 'PS_MAG_I', 'PS_MAG_I_ERR', 
                               'PS_MAG_R', 'PS_MAG_R_ERR', 'PS_MAG_Y', 'PS_MAG_Y_ERR', 'PS_MAG_Z', 'PS_MAG_Z_ERR',
                                'NIRCAT_RA',  'NIRCAT_DEC',  'NIRCAT_ID', 'NIRCAT_MAG_H', 'NIRCAT_MAG_H_ERR', 
                               'NIRCAT_MAG_J', 'NIRCAT_MAG_J_ERR', 'NIRCAT_MAG_K', 'NIRCAT_MAG_K_ERR',
                               'OPTCAT_RA', 'OPTCAT_DEC',   'OPTCAT_ID', 'OPTCAT_MAG_B', 'OPTCAT_MAG_B_ERR'
                              ]

FITS_table_from_cat.reset_index()
#survey identifier
FITS_table_from_cat['TARGSRVY']='WS2021A1-003'

#CNAME is set by WEAVE
FITS_table_from_cat['CNAME']=''

#optional personal name for programme/catalogue
FITS_table_from_cat['TARGPROG']='GW-galaxy'
FITS_table_from_cat['TARGCAT']='GW-galaxy'

#T for target, GALAXY
FITS_table_from_cat['TARGUSE']='T'
FITS_table_from_cat['TARGCLASS']='GALAXY'

# PROGTEMP for MOS LR 3x1020s 60min OB
FITS_table_from_cat['PROGTEMP']='11331'

#OBSTEMP for max airmass=2.4, moon dist=90 deg, 
#max sky brightness=20.5, max seeing = 1.3'', transparency=0.8
FITS_table_from_cat['OBSTEMP']='GAFAD'

#TARGNAME is the same as unique id
FITS_table_from_cat['TARGNAME']=FITS_table_from_cat['TARGID']

#TARGPRIO = 10.0 for the moment
FITS_table_from_cat['TARGPRIO']=10.0

def DR (df):
    if df['source'] == 'GLADE' :
        return 'V2'
    else:
        return ''
def glade (df):
    if df['source'] == 'GLADE' :
        return 'GLADE'
    else:
        return ''
    
def gaia (df):
    if df['GAIA_ID']!='':
        return 'eDR3'
    else:
        return ''

def PS (df):
    if df['PS_ID']!='':
        return '2'
    else:
        return ''
    
# ID columns are strings
FITS_table_from_cat['GAIA_ID']=FITS_table_from_cat['GAIA_ID'].astype(str)
FITS_table_from_cat['PS_ID']=FITS_table_from_cat['PS_ID'].astype(str)
FITS_table_from_cat['OPTCAT_ID']=FITS_table_from_cat['OPTCAT_ID'].astype(str)
FITS_table_from_cat['NIRCAT_ID']=FITS_table_from_cat['NIRCAT_ID'].astype(str)

# Null values for ID columns should be empty strings
FITS_table_from_cat.loc[FITS_table_from_cat.GAIA_ID == 'nan', ['GAIA_ID']] = ''
FITS_table_from_cat.loc[FITS_table_from_cat.NIRCAT_ID == 'nan', ['NIRCAT_ID']] = ''
FITS_table_from_cat.loc[FITS_table_from_cat.OPTCAT_ID == 'nan', ['OPTCAT_ID']] = ''
FITS_table_from_cat.loc[FITS_table_from_cat.PS_ID == 'nan', ['PS_ID']] = ''

# only give NIRCAT/OPTCAT columns for GLADE galaxies.
FITS_table_from_cat['NIRCAT_DR']=total_cut_df.apply (lambda total_cut_df: DR(total_cut_df), axis=1)
FITS_table_from_cat['OPTCAT_DR']=total_cut_df.apply (lambda total_cut_df: DR(total_cut_df), axis=1)
FITS_table_from_cat['NIRCAT']=total_cut_df.apply (lambda total_cut_df: glade(total_cut_df), axis=1)
FITS_table_from_cat['OPTCAT']=total_cut_df.apply (lambda total_cut_df: glade(total_cut_df), axis=1)

#Only give Gaia/PS columns for sources crossmatched with either.
FITS_table_from_cat['GAIA_DR']=FITS_table_from_cat.apply (lambda FITS_table_from_cat: gaia(FITS_table_from_cat), axis=1)
FITS_table_from_cat['PS_DR']=FITS_table_from_cat.apply (lambda FITS_table_from_cat: PS(FITS_table_from_cat), axis=1)

# PS nulls are -999 - set to nan
FITS_table_from_cat=FITS_table_from_cat.replace(-999.000000, np.nan)


# Set APS / IFU columns to the default by setting them to null

FITS_table_from_cat['APS_PPXF_MOM']=-1
FITS_table_from_cat['APS_FLAG'] =''
FITS_table_from_cat['IFU_SPAXEL']=''
FITS_table_from_cat['APS_Z']=np.nan
FITS_table_from_cat['APS_TEMPL_LIB']=''
FITS_table_from_cat['APS_GAND_EBV']=-1
FITS_table_from_cat['APS_GAND_ERR']=-1
FITS_table_from_cat['APS_TEMPL_LIB_NORM']=-1
FITS_table_from_cat['APS_IFU_TSSL_MIN_SNR'] =np.nan
FITS_table_from_cat['IFU_PA'] =np.nan
FITS_table_from_cat['IFU_DITHER'] =-1
FITS_table_from_cat['APS_WL_MIN'] =np.nan
FITS_table_from_cat['APS_IFU_TSSL_TYPE'] =''
FITS_table_from_cat['GENCAT_INFO'] =''
FITS_table_from_cat['APS_SSP_NUM_WLKR'] =-1
FITS_table_from_cat['APS_LS_NUM_MC'] =-1
FITS_table_from_cat['APS_PPXF_DEG_MULT'] =-1
FITS_table_from_cat['APS_GAND_MODE'] =-1
FITS_table_from_cat['APS_GAND_RED2'] =np.nan
FITS_table_from_cat['APS_PPXF_NUM_MC'] =-1
FITS_table_from_cat['APS_IFU_SRC_DEC']=np.nan
FITS_table_from_cat['APS_IFU_SRC_ID'] =''
FITS_table_from_cat['APS_LS_RES'] =np.nan
FITS_table_from_cat['APS_IFU_TSSL_COVAR'] =-1
FITS_table_from_cat['APS_SSP_NUM_CHAIN'] =-1
FITS_table_from_cat['HEALPIX'] =-1
FITS_table_from_cat['APS_LS_MODE'] =-1
FITS_table_from_cat['APS_WL_MAX'] =np.nan
FITS_table_from_cat['APS_IFU_MASK'] =-1
FITS_table_from_cat['APS_PPXF_DEG_ADD'] =-1
FITS_table_from_cat['APS_PPXF_WL_MAX'] =np.nan
FITS_table_from_cat['APS_IFU_TSSL_TARG_SNR'] =np.nan
FITS_table_from_cat['APS_GAND_RED1'] =np.nan
FITS_table_from_cat['APS_IFU_SRC_RA'] =np.nan
FITS_table_from_cat['APS_SIGMA'] =np.nan
FITS_table_from_cat['APS_PPXF_WL_MIN']=np.nan

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

## If we don't have Gaia RA/Dec, transform PS1/GLADE to J2016 and put this into Gaia_RA, Gaia_dec columns

In [458]:
# Take PS or GLADE ra,dec,epoch
for t in range(0,len(FITS_table_from_cat)):
    if FITS_table_from_cat['GAIA_ID'][t]=='':
        if FITS_table_from_cat['PS_ID'][t]!='':
            FITS_table_from_cat['GAIA_RA'][t]=FITS_table_from_cat['PS_RA'][t]
            FITS_table_from_cat['GAIA_DEC'][t]=FITS_table_from_cat['PS_DEC'][t]
            FITS_table_from_cat['GAIA_EPOCH'][t]=2016.0
        else:
            FITS_table_from_cat['GAIA_RA'][t]=FITS_table_from_cat['OPTCAT_RA']
            FITS_table_from_cat['GAIA_DEC'][t]=FITS_table_from_cat['OPTCAT_DEC']
            FITS_table_from_cat['GAIA_EPOCH'][t]=2016.0
            

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A va

ValueError: No axis named 1 for object type Series

## Fill in source of Mag G, Mag R, Mag I columns

In [459]:
# give a list of the columns where the Mag_GRI column came from. 
mag_source_g=[]
mag_source_r=[]
mag_source_i=[]

for t in range(0,len(total_cut_df)):
    if total_cut_df['source'][t]=='GLADE':
        mag_source_g.append('PS_MAG_G')
        mag_source_r.append('PS_MAG_R')
        mag_source_i.append('PS_MAG_I')
    else:
        mag_source_g.append('PS_MAG_G')
        mag_source_r.append('PS_MAG_R')
        mag_source_i.append('PS_MAG_I')

ext0_header['MAG_G_CM']='|'.join(mag_source_g)
ext0_header['MAG_R_CM']='|'.join(mag_source_r)
ext0_header['MAG_I_CM']='|'.join(mag_source_i)

# Set TARGPRIO based on magnitude

# Study new dataframe

In [460]:
pd.set_option('display.max_columns', None)
FITS_table_from_cat=FITS_table_from_cat[template_column_names]
display(FITS_table_from_cat)

,CNAME,TARGSRVY,TARGPROG,TARGCAT,TARGID,TARGNAME,TARGPRIO,TARGUSE,TARGCLASS,PROGTEMP,OBSTEMP,GAIA_ID,GAIA_DR,GAIA_RA,GAIA_DEC,GAIA_EPOCH,GAIA_PMRA,GAIA_PMRA_ERR,GAIA_PMDEC,GAIA_PMDEC_ERR,GAIA_PARAL,GAIA_PARAL_ERR,HEALPIX,IFU_SPAXEL,IFU_PA,IFU_DITHER,MAG_G,MAG_G_ERR,MAG_R,MAG_R_ERR,MAG_I,MAG_I_ERR,GAIA_MAG_G,GAIA_MAG_G_ERR,GAIA_MAG_BP,GAIA_MAG_BP_ERR,GAIA_MAG_RP,GAIA_MAG_RP_ERR,APS_WL_MIN,APS_WL_MAX,APS_Z,APS_SIGMA,APS_TEMPL_LIB,APS_TEMPL_LIB_NORM,APS_PPXF_WL_MIN,APS_PPXF_WL_MAX,APS_PPXF_MOM,APS_PPXF_DEG_ADD,APS_PPXF_DEG_MULT,APS_PPXF_NUM_MC,APS_GAND_MODE,APS_GAND_ERR,APS_GAND_RED1,APS_GAND_RED2,APS_GAND_EBV,APS_LS_MODE,APS_LS_RES,APS_LS_NUM_MC,APS_SSP_NUM_WLKR,APS_SSP_NUM_CHAIN,APS_IFU_MASK,APS_IFU_TSSL_TYPE,APS_IFU_TSSL_TARG_SNR,APS_IFU_TSSL_MIN_SNR,APS_IFU_TSSL_COVAR,APS_IFU_SRC_ID,APS_IFU_SRC_RA,APS_IFU_SRC_DEC,APS_FLAG,NIRCAT,NIRCAT_DEC,NIRCAT_DR,NIRCAT_ID,NIRCAT_MAG_H,NIRCAT_MAG_H_ERR,NIRCAT_MAG_J,NIRCAT_MAG_J_ERR,NIRCAT_MAG_K,NIRCAT_MAG_K_ERR,NIRCAT_RA,OPTCAT,OPTCAT_DEC,OPTCAT_DR,OPTCAT_ID,OPTCAT_MAG_B,OPTCAT_MAG_B_ERR,OPTCAT_RA,PS_DEC,PS_DR,PS_ID,PS_MAG_G,PS_MAG_G_ERR,PS_MAG_I,PS_MAG_I_ERR,PS_MAG_R,PS_MAG_R_ERR,PS_MAG_Y,PS_MAG_Y_ERR,PS_MAG_Z,PS_MAG_Z_ERR,PS_RA
0,,WS2021A1-003,GW-galaxy,GW-galaxy,191213_10004,191213_10004,10.0,T,GALAXY,11331,GAFAD,,,252.269439,-2.694974,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,-1,,NaN,-1,20.065201,0.063170,19.011101,0.051448,18.574600,0.006902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,NaN,-1,-1,-1,-1,,NaN,NaN,-1,,NaN,NaN,,,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,,NaN,NaN,NaN,-2.694974,2,1.04763e+17,20.065201,0.063170,18.574600,0.006902,19.011101,0.051448,NaN,NaN,18.162901,0.088184,252.269439
1,,WS2021A1-003,GW-galaxy,GW-galaxy,191213_10012,191213_10012,10.0,T,GALAXY,11331,GAFAD,,,250.872067,-2.685312,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,-1,,NaN,-1,19.807699,0.046431,18.963600,0.018857,18.302300,0.013357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,NaN,-1,-1,-1,-1,,NaN,NaN,-1,,NaN,NaN,,,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,,NaN,NaN,NaN,-2.685312,2,1.04773e+17,19.807699,0.046431,18.302300,0.013357,18.963600,0.018857,17.740801,0.031454,17.942699,0.022088,250.872067
2,,WS2021A1-003,GW-galaxy,GW-galaxy,191213_10015,191213_10015,10.0,T,GALAXY,11331,GAFAD,,,252.275868,-2.642753,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,-1,,NaN,-1,20.273199,0.071873,19.482500,0.028331,19.077000,0.045716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,NaN,-1,-1,-1,-1,,NaN,NaN,-1,,NaN,NaN,,,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,,NaN,NaN,NaN,-2.642753,2,1.04823e+17,20.273199,0.071873,19.077000,0.045716,19.482500,0.028331,18.398600,0.192583,18.994301,0.057580,252.275868
3,,WS2021A1-003,GW-galaxy,GW-galaxy,191213_10016,191213_10016,10.0,T,GALAXY,11331,GAFAD,,,251.347515,-2.637620,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,-1,,NaN,-1,19.874201,0.035306,19.014200,0.027404,18.611200,0.018717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,NaN,-1,-1,-1,-1,,NaN,NaN,-1,,NaN,NaN,,,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,,NaN,NaN,NaN,-2.637620,2,1.04833e+17,19.874201,0.035306,18.611200,0.018717,19.014200,0.027404,18.295099,0.039521,18.461700,0.038863,251.347515
4,,WS2021A1-003,GW-galaxy,GW-galaxy,191213_10017,191213_10017,10.0,T,GALAXY,11331,GAFAD,,,252.440333,-2.638061,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,-1,,NaN,-1,19.335800,0.041282,18.586500,0.019701,18.171101,0.013297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,NaN,-1,-1,-1,-1,,NaN,NaN,-1,,NaN,NaN,,,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,,NaN,NaN,NaN,-2.638061,2,1.04833e+17,19.335800,0.041282,18.171101,0.013297,18.586500,0.019701,17.934099,0.047953,18.128500,0.034402,252.440333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.